In [5]:
# !pip install pyfim
# !pip install pandas
# !pip install matplotlibx 
# !pip install unlzw
# !pip install pyfim

ERROR: Could not find a version that satisfies the requirement matplotlibx (from versions: none)
ERROR: No matching distribution found for matplotlibx


In [7]:
import matplotlib.pyplot as plt
from sklearn import tree
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from unlzw import unlzw
import csv, re
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pyarc import TransactionDB
from pyarc.algorithms import (
    top_rules,
    createCARs,
    M1Algorithm,
    M2Algorithm
)
import Orange
from sklearn.model_selection import train_test_split
import pandas as pd
from Orange.data.pandas_compat import table_from_frame,table_to_frame


from yarc.Structure import TransactionDB
from yarc.Mine_Classi_Alg.generating_CARS import ClassAssocationRule, Antecedent, Consequent, top_rules, CARlist
from yarc.Mine_Classi_Alg.m2classi import M2Classi
from yarc.Mine_Classi_Alg.predictor import Predictor
from yarc import CBA #unable to import it 
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

In [8]:
def discretiseRule(X):
    temp = Orange.data.Table(X)
    disc = Orange.preprocess.Discretize()
    disc.method = Orange.preprocess.discretize.EqualFreq(n=3)
    d_temp = disc(temp)
    X= table_to_frame(d_temp)
    return X

In [9]:
def runCBA(X,y,target):
    X = discretiseRule(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=25, test_size=0.2, stratify = y)#25
    train = pd.concat([X_train, y_train], axis=1)
    test=pd.concat([X_test, y_test], axis=1)
    txns = TransactionDB.from_DataFrame(train, target=target) #../../
    txnstest = TransactionDB.from_DataFrame(test, target=target) #../../

    cba = CBA()
    cba.fit(txns)
    cba.target_class
    cba.pre.rules
    
    cbaTrainAcc=cba.rule_model_accuracy(txns)
    cbaTestAcc=cba.rule_model_accuracy(txnstest)
    # get the best association rules
    rules = top_rules(txns.string_representation)

    # convert them to class association rules
    cars = CARlist(rules)
    

    predictor = M2Classi(cars, txns).build()
    topKRules = predictor.rules
    accuracy = predictor.test_transactions(txnstest)
    #print(cba.pre.rules)
    print("CBA Train Accuracy:",str(cbaTrainAcc))
    print("CBA Test Accuracy:",str(cbaTestAcc))
    print("Top K Test Accuracy:",str(accuracy))
    #print(cba.pre.rules)
    print(topKRules)
    #print(rules)

In [10]:
# Load the iris dataset
# iris = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header = None)
# iris = pd.read_csv('iris.data', header = None)
# iris.columns=["sepallength","petalwidth","sepalwidth","petallength","class"]
# iris.to_csv("iris.csv", index = None, header = True)
iris = pd.read_csv('iris.csv',index= None, header = True)

#Load the waveform dataset #CART decision tree algorithm: 72% (Information from the .names file)
with open('waveform.data.Z', 'rb') as fh:
    compressed_data = fh.read()
    uncompressed_data = unlzw(compressed_data)
data = uncompressed_data.decode('utf-8').splitlines()
with open("wave.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter = '\t')
    for line in data:
        writer.writerow(re.split('\s+',line))
wave = pd.read_csv('wave.csv', header = None)
wave.to_csv("wave.csv", index = None, header = True)
wave_labels = []
for i in range(21):
  wave_labels.append('x'+str(i))
wave_labels.append('class')
wave.columns=wave_labels
wave.to_csv("wave.csv", index = None, header = True)

#Load the breast cancer dataset
breastCancer = pd.read_csv("breast-cancer.data")
breastCancer.columns=["class","age", "menopause", "tumor-size", "inv-nodes", "node-caps","deg-malig","breast","breast-quad","irradiat"]
breastCancer.to_csv("breastCancer.csv", index = None, header = True)
#wave.to_csv("breastCancer.csv", index = None, header = True)


#Load the german dataset
german = pd.read_csv('german.data', header = None)
german.head()
german_columns=["checkAccStatus", "durationMth", "credHist", "purpose", "credAmt","savAccBond","emplySince","instRate","personalStatSex","otherDebtGuar","presResSince","prpty","age(years)","otherInstallPlans","housing","numExistCreds","job","numPplMaintain","telephone","frgnWorker","goodBad"]
german = german[0].str.split(" ", n = 20, expand = True)
german.columns = german_columns
german.to_csv("german.csv", index = None, header = True)
#german.columns = german_columns

#Load the heart dataset
heart = pd.read_csv('heart.dat', header = None)
heart = heart[0].str.split(" ", n = 13, expand = True)
heart.columns=["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal","class"]
heart_columns= ["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal"]
heart.to_csv("heart.csv", index = None, header = True)


#Load the kaggle dataset
#stroke
stroke = pd.read_csv("healthcare-dataset-stroke-data.csv", sep=",")

#Heart Attack
campusPlacement = pd.read_csv("Placement_Data_Full_Class.csv", sep=";", index_col=0)

TypeError: read_csv() got an unexpected keyword argument 'index'

Discretising the rules

In [11]:
#remove this cell before submission

#function to export a CSV with the discretised values
# def createCSV(X,y):
#     temp = Orange.data.Table(X)
#     disc = Orange.preprocess.Discretize()
#     disc.method = Orange.preprocess.discretize.EqualFreq(n=3)
#     d_temp = disc(temp)
#     X= table_to_frame(d_temp)
#     frames = [X, y]
#     result = pd.concat(frames, axis=1)
#     result.columns=["sepallength","petalwidth","sepalwidth","petallength","class"]
#     result.to_csv("newIris.csv", index=None, header = True)

# Iris

In [12]:
read_file = pd.read_csv ('iris.csv') #discretised iris from their database #currently its the pure one without discretising
read_file = read_file.dropna()
X=read_file[["sepallength","petalwidth","sepalwidth","petallength"]]
y=read_file[["class"]]
runCBA(X,y,"class")

CBA Train Accuracy: 0.9666666666666667
CBA Test Accuracy: 0.9
Top K Test Accuracy: 0.9
[CAR {Feature 3=2.45 - 4.85,Feature 4=0.8 - 1.55} => {class=Iris-versicolor} sup: 0.30 conf: 1.00 len: 3, id: 1968, CAR {Feature 3=2.45 - 4.85,Feature 1=≥ 6.25} => {class=Iris-versicolor} sup: 0.08 conf: 1.00 len: 3, id: 1595, CAR {Feature 3=2.45 - 4.85,Feature 2=≥ 3.15} => {class=Iris-versicolor} sup: 0.03 conf: 1.00 len: 3, id: 1363, CAR {Feature 2=2.85 - 3.15,Feature 4=≥ 1.55,Feature 1=5.45 - 6.25} => {class=Iris-virginica} sup: 0.02 conf: 1.00 len: 4, id: 1777, CAR {Feature 2=< 2.85,Feature 3=2.45 - 4.85,Feature 4=≥ 1.55} => {class=Iris-virginica} sup: 0.02 conf: 1.00 len: 4, id: 2223, CAR {Feature 1=≥ 6.25,Feature 3=≥ 4.85} => {class=Iris-virginica} sup: 0.24 conf: 0.97 len: 3, id: 1924, CAR {Feature 3=≥ 4.85} => {class=Iris-virginica} sup: 0.31 conf: 0.95 len: 2, id: 1951]


C:\Users\theco\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:2: OrangeDeprecationWarning: Omitting domain in a call to Table(X, Y, metas), is deprecated and will be removed. Call Table.from_numpy(None, X, Y, metas) instead.
  


# Wave

In [13]:
read_file = pd.read_csv ('wave.csv')
read_file = read_file.dropna()
X=read_file[['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20']]
y=read_file[["class"]]
runCBA(X,y,"class")

C:\Users\theco\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:2: OrangeDeprecationWarning: Omitting domain in a call to Table(X, Y, metas), is deprecated and will be removed. Call Table.from_numpy(None, X, Y, metas) instead.
  


CBA Train Accuracy: 0.7725
CBA Test Accuracy: 0.764
Top K Test Accuracy: 0.797
[CAR {Feature 11=≥ 4.025,Feature 07=≥ 3.615} => {class=1} sup: 0.06 conf: 0.91 len: 3, id: 97884, CAR {Feature 06=≥ 2.785,Feature 17=≥ 2.055} => {class=0} sup: 0.03 conf: 0.91 len: 3, id: 93122, CAR {Feature 05=≥ 2.035,Feature 17=≥ 2.055} => {class=0} sup: 0.04 conf: 0.90 len: 3, id: 94148, CAR {Feature 17=≥ 2.055,Feature 13=< 1.945} => {class=0} sup: 0.04 conf: 0.90 len: 3, id: 93143, CAR {Feature 11=< 2.445,Feature 07=1.585 - 3.615} => {class=0} sup: 0.07 conf: 0.89 len: 3, id: 48576, CAR {Feature 11=≥ 4.025,Feature 06=≥ 2.785} => {class=1} sup: 0.05 conf: 0.89 len: 3, id: 95548, CAR {Feature 16=≥ 2.775,Feature 11=≥ 4.025} => {class=2} sup: 0.05 conf: 0.89 len: 3, id: 77502, CAR {Feature 15=≥ 3.575,Feature 06=≥ 2.785} => {class=0} sup: 0.01 conf: 0.89 len: 3, id: 49020, CAR {Feature 16=≥ 2.775,Feature 13=< 1.945} => {class=0} sup: 0.03 conf: 0.89 len: 3, id: 80345, CAR {Feature 15=≥ 3.575,Feature 13=< 1.94

# Heart

In [14]:
read_file = pd.read_csv ('heart.csv')
read_file = read_file.dropna()
X=read_file[["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal"]]
y=read_file[["class"]]

runCBA(X,y,"class")

C:\Users\theco\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:2: OrangeDeprecationWarning: Omitting domain in a call to Table(X, Y, metas), is deprecated and will be removed. Call Table.from_numpy(None, X, Y, metas) instead.
  


CBA Train Accuracy: 0.9259259259259259
CBA Test Accuracy: 0.8148148148148148
Top K Test Accuracy: 0.7777777777777778
[CAR {Feature 04=< 121,Feature 02=< 0.5} => {class=1} sup: 0.11 conf: 1.00 len: 3, id: 139101, CAR {Feature 05=< 226.5,Feature 03=< 2.5} => {class=1} sup: 0.09 conf: 1.00 len: 3, id: 145391, CAR {Feature 09=≥ 0.5,Feature 12=≥ 1.5} => {class=2} sup: 0.08 conf: 1.00 len: 3, id: 148071, CAR {Feature 03=≥ 3.5,Feature 11=≥ 2.5} => {class=2} sup: 0.03 conf: 1.00 len: 3, id: 150676, CAR {Feature 08=≥ 161.5,Feature 03=2.5 - 3.5} => {class=1} sup: 0.12 conf: 0.96 len: 3, id: 144137, CAR {Feature 03=2.5 - 3.5,Feature 02=< 0.5} => {class=1} sup: 0.12 conf: 0.96 len: 3, id: 144661, CAR {Feature 03=≥ 3.5,Feature 12=0.5 - 1.5} => {class=2} sup: 0.11 conf: 0.96 len: 3, id: 146462, CAR {Feature 07=≥ 1.5,Feature 12=≥ 1.5} => {class=2} sup: 0.11 conf: 0.96 len: 3, id: 147832, CAR {Feature 11=< 1.5,Feature 02=< 0.5} => {class=1} sup: 0.17 conf: 0.95 len: 3, id: 138668, CAR {Feature 08=< 14

# Breast Cancer

In [15]:
#error: cannot convert string to float
from sklearn.preprocessing import OrdinalEncoder
read_file = pd.read_csv ('breastCancer.csv')
read_file = read_file.dropna()
X=read_file[["menopause", "age", "tumor-size", "inv-nodes", "node-caps","deg-malig","breast","breast-quad","irradiat"]]
#encoding the normal data
for col in ["menopause","node-caps","breast","breast-quad","irradiat"]:
   X[col] = LabelEncoder().fit_transform(X[col])
#encoding the ordinal data #Ordinal Encoder not working so i just hardcoded first
ordinalData = ["age","tumor-size","inv-nodes"]
for i in ordinalData:
   columns = X[i].unique()
   columns.sort() #sorting the labels
   for j in range(len(columns)):
      X.loc[X[i]==columns[j],i] = j
y=read_file[["class"]]
runCBA(X,y,"class")

C:\Users\theco\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\theco\AppData\Local\Programs\Python\Python37-32\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\theco\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:2: OrangeDeprecationWarning: Omitting domain in a call to Table(X, Y, met

CBA Train Accuracy: 0.8201754385964912
CBA Test Accuracy: 0.6491228070175439
Top K Test Accuracy: 0.6666666666666666
[CAR {Feature 1=< 0.5,Feature 5=< 0.5} => {class=no-recurrence-events} sup: 0.02 conf: 1.00 len: 3, id: 160082, CAR {Feature 1=< 0.5,Feature 6=< 1.5} => {class=no-recurrence-events} sup: 0.12 conf: 0.96 len: 3, id: 155882, CAR {Feature 6=< 1.5,Feature 3=< 3.5} => {class=no-recurrence-events} sup: 0.11 conf: 0.92 len: 3, id: 156020, CAR {Feature 6=< 1.5,Feature 8=≥ 3.5} => {class=no-recurrence-events} sup: 0.04 conf: 0.91 len: 3, id: 158241, CAR {Feature 2=2.5 - 3.5,Feature 6=< 1.5} => {class=no-recurrence-events} sup: 0.08 conf: 0.90 len: 3, id: 156087, CAR {Feature 2=2.5 - 3.5,Feature 3=< 3.5} => {class=no-recurrence-events} sup: 0.11 conf: 0.90 len: 3, id: 155162, CAR {Feature 5=0.5 - 1.5,Feature 3=< 3.5} => {class=no-recurrence-events} sup: 0.30 conf: 0.90 len: 3, id: 154594, CAR {Feature 8=< 2.5,Feature 6=< 1.5} => {class=no-recurrence-events} sup: 0.10 conf: 0.88 le

# German

In [16]:
#Can look into whether there are any ordinal rules to encode
read_file = pd.read_csv ('german.csv')
read_file = read_file.dropna()
X=read_file[["checkAccStatus", "durationMth", "credHist", "purpose", "credAmt","savAccBond","emplySince","instRate","personalStatSex","otherDebtGuar","presResSince","prpty","age(years)","otherInstallPlans","housing","numExistCreds","job","numPplMaintain","telephone","frgnWorker"]]
for col in ['checkAccStatus','credHist','purpose','savAccBond','emplySince','personalStatSex','otherDebtGuar','prpty','otherInstallPlans','housing','job','telephone','frgnWorker']:
    X[col] = LabelEncoder().fit_transform(X[col])
y=read_file[["goodBad"]]
runCBA(X,y,"goodBad")

C:\Users\theco\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\theco\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:2: OrangeDeprecationWarning: Omitting domain in a call to Table(X, Y, metas), is deprecated and will be removed. Call Table.from_numpy(None, X, Y, metas) instead.
  


CBA Train Accuracy: 0.78375
CBA Test Accuracy: 0.7
Top K Test Accuracy: 0.73
[CAR {Feature 09=1.5 - 2.5,Feature 20=≥ 0.5} => {goodBad=1} sup: 0.02 conf: 1.00 len: 3, id: 252332, CAR {Feature 01=≥ 2.5,Feature 11=2.5 - 3.5} => {goodBad=1} sup: 0.05 conf: 0.96 len: 3, id: 217763, CAR {Feature 01=≥ 2.5,Feature 07=≥ 2.5} => {goodBad=1} sup: 0.18 conf: 0.95 len: 3, id: 172283, CAR {Feature 03=≥ 2.5,Feature 06=≥ 3.5} => {goodBad=1} sup: 0.07 conf: 0.95 len: 3, id: 209272, CAR {Feature 06=≥ 3.5,Feature 11=2.5 - 3.5} => {goodBad=1} sup: 0.02 conf: 0.94 len: 3, id: 219973, CAR {Feature 10=≥ 1.5,Feature 02=< 12.5} => {goodBad=1} sup: 0.02 conf: 0.94 len: 3, id: 241908, CAR {Feature 01=≥ 2.5,Feature 06=≥ 3.5} => {goodBad=1} sup: 0.09 conf: 0.94 len: 3, id: 209213, CAR {Feature 01=≥ 2.5,Feature 05=< 1553.5} => {goodBad=1} sup: 0.12 conf: 0.93 len: 3, id: 178751, CAR {Feature 05=1553.5 - 3362.5,Feature 06=≥ 3.5} => {goodBad=1} sup: 0.05 conf: 0.93 len: 3, id: 209834, CAR {Feature 01=≥ 2.5,Feature 14

# University Student Placement (To classify as either placed or not placed)

In [17]:
read_file = pd.read_csv ('Placement_data_Full_Class.csv')
read_file = read_file.dropna()
X = read_file[['sl_no', 'gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'hsc_s', 'degree_p', 'degree_t', 'workex', 'etest_p', 'specialisation', 'mba_p', 'salary']]
#creating a function to encode categorical features into numerical
for col in ['gender','ssc_b','hsc_b','hsc_s','degree_t','workex','specialisation']:
    X[col] = LabelEncoder().fit_transform(X[col])
y=read_file[['status']]
runCBA(X,y,"status")

C:\Users\theco\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\theco\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:2: OrangeDeprecationWarning: Omitting domain in a call to Table(X, Y, metas), is deprecated and will be removed. Call Table.from_numpy(None, X, Y, metas) instead.
  


CBA Train Accuracy: 0.7177914110429447
CBA Test Accuracy: 0.5370370370370371
Top K Test Accuracy: 0.5370370370370371
[CAR {Feature 05=≥ 73.1,Feature 14=245000 - 288500} => {status=Placed} sup: 0.07 conf: 0.92 len: 3, id: 260829]


In [18]:
stroke = stroke.dropna()
X=stroke[["gender", "age", "heart_disease", "ever_married", "work_type","Residence_type","avg_glucose_level","bmi","smoking_status"]]
#encoding the normal data
for col in ["gender","ever_married","work_type","Residence_type"]:
   X[col] = LabelEncoder().fit_transform(X[col])
#encoding the ordinal data #Ordinal Encoder not working so i just hardcoded first
ordinalData = ["smoking_status"]
for i in ordinalData:
   smokingStatus = ["never smoked","formerly smoked","smokes",] #need to think about how to encode unknown
   smokingStatus = ["Unknown","never smoked","formerly smoked","smokes"] #need to think about how to encode unknown
   for j in range(len(smokingStatus)):
      X.loc[X[i]==smokingStatus[j],i] = int(j)
y=stroke[["stroke"]]
runCBA(X,y,"stroke") #for some reason this cannot work cuz it keep going out of range

NameError: name 'stroke' is not defined

In [ ]:
X.tail()

In [ ]:
cba.predict_probability(txns)

In [ ]:
ClassAssocationRule(Antecedent({}), Consequent("class", "default_class1"), 0.1, 0.2)

In [ ]:
cba.rule_model_accuracy(txns)

In [ ]:
cba.rule_model_accuracy(txnstest)

In [ ]:
# get the best association rules
rules = top_rules(txns.string_representation)

# convert them to class association rules
cars = CARlist(rules)
predictor = M2Classi(cars, txns).build()


accuracy = predictor.test_transactions(txnstest)


In [ ]:
print(accuracy)

In [ ]:
read_file = pd.read_csv ('heart.csv')
read_file = read_file.dropna()
X=read_file[ ["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal"]]
y=read_file[["class"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14, test_size=0.2, stratify = y)#25
train = pd.concat([X_train, y_train], axis=1)
test=pd.concat([X_test, y_test], axis=1)
txns = TransactionDB.from_DataFrame(train, target="class") #../../
txnstest = TransactionDB.from_DataFrame(test, target="class") #../../

cba = CBA()

In [ ]:
cba.fit(txns)
cba.target_class
cba.pre.rules

In [ ]:
cba.predict_probability(txns)

In [ ]:
ClassAssocationRule(Antecedent({}), Consequent("class", "default_class1"), 0.1, 0.2)

In [ ]:
cba.rule_model_accuracy(txns)

In [ ]:
cba.rule_model_accuracy(txnstest)

In [ ]:
# get the best association rules
rules = top_rules(txns.string_representation)

# convert them to class association rules
cars = CARlist(rules)

predictor = M2Classi(cars, txns).build()


accuracy = predictor.test_transactions(txnstest)
print(accuracy)